In [11]:
import pandas as pd
import numpy as np
import time
import dlib
import cv2
from imutils import face_utils
from scipy.spatial import distance
# from playsound import playsound
import pygame

In [12]:
def drowsiness_alert():
    for i in range (2):
        playsound('drowsiness_detected.mp3')
        playsound('alert.wav')
        time.sleep(1)

In [13]:
def yawn_alert():
    for i in range (2):
        playsound('yawn_alert.mp3')
        time.sleep(1.5)

In [14]:
def draw_label(img, text, pos, bg_color):
    text_size= cv2.getTextSize(text, cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, cv2.FILLED)
    
    end_x= pos[0] + text_size[0][0] + 90
    end_y= pos[1] + text_size[0][1] +10
    pos=list(pos)
    pos[0]-=10
    pos[1]-=15

    cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
    pos[0]+=8
    pos[1]+=32
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)

In [15]:
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
detector = dlib.get_frontal_face_detector()

In [16]:
EYE_ASPECT_RATIO_THRESHOLD = 0.22

EYE_ASPECT_RATIO_CONSEC_FRAMES = 10

f_cnt = 0

pygame.mixer.init()

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

In [17]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])

    ear = (A+B) / (2*C)
    return ear

def avg_aspect_ratio (frame):
    faces = detector(frame,0)
#     for face in faces:
    try:
        face=faces[0]
        shape = predictor(frame, face)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]

        leftEyeAspectRatio = eye_aspect_ratio(leftEye)
        rightEyeAspectRatio = eye_aspect_ratio(rightEye)

        eyeAspectRatio = (leftEyeAspectRatio + rightEyeAspectRatio) / 2
        return eyeAspectRatio
    except:
        return 0.5

In [18]:
yawn_status = False 
d_timer=5

In [19]:
def get_landmarks(im):
    rects = detector(im, 1)

    if len(rects) > 1:
        return "error"
    if len(rects) == 0:
        return "error"
    return np.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])


def annotate_landmarks(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

def top_lip(landmarks):
    top_lip_pts = []
    for i in range(50,53):
        top_lip_pts.append(landmarks[i])
    for i in range(61,64):
        top_lip_pts.append(landmarks[i])
    top_lip_all_pts = np.squeeze(np.asarray(top_lip_pts))
    top_lip_mean = np.mean(top_lip_pts, axis=0)
    return int(top_lip_mean[:,1])

def bottom_lip(landmarks):
    bottom_lip_pts = []
    for i in range(65,68):
        bottom_lip_pts.append(landmarks[i])
    for i in range(56,59):
        bottom_lip_pts.append(landmarks[i])
    bottom_lip_all_pts = np.squeeze(np.asarray(bottom_lip_pts))
    bottom_lip_mean = np.mean(bottom_lip_pts, axis=0)
    return int(bottom_lip_mean[:,1])

def mouth_open(image):
    landmarks = get_landmarks(image)
    
    if landmarks == "error":
        return image, 0
    
    image_with_landmarks = annotate_landmarks(image, landmarks)
    top_lip_center = top_lip(landmarks)
    bottom_lip_center = bottom_lip(landmarks)
    lip_distance = abs(top_lip_center - bottom_lip_center)
    return image_with_landmarks, lip_distance

    #cv2.imshow('Result', image_with_landmarks)
    #cv2.imwrite('image_with_landmarks.jpg',image_with_landmarks)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

In [20]:
cap = cv2.VideoCapture(0)
f_cnt = 0

while True:
    ret, frame = cap.read()   
    image_landmarks, lip_distance = mouth_open(frame)
    prev_yawn_status = yawn_status  
    
    frame1 = cv2.flip(frame,1)
    frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    ear=avg_aspect_ratio(frame1)

    print(f_cnt)
    
    if lip_distance > 30:
        draw_label(frame, "Yawning", (0,30), (0, 0, 255))
        yawn_status = True 
        
        
    else:
        draw_label(frame, "Not Yawning", (0,30), (0, 255, 0))
        yawn_status = False 
        
         
    if prev_yawn_status == True and yawn_status == False:
        pygame.mixer.music.load("yawn_alert.mp3")
        pygame.mixer.music.play()

    if(ear < EYE_ASPECT_RATIO_THRESHOLD and yawn_status==False):
        f_cnt += 1
    else:
        f_cnt=0
    
    
    if f_cnt>5:
        if d_timer>0:
            cv2.putText(frame,str(d_timer),(110,470), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 20, (129, 133, 137), 12, cv2.LINE_AA)
            d_timer-=1
    else:
        d_timer=5
    
        #(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
    
    
#     print(ear, f_cnt)

    if f_cnt == EYE_ASPECT_RATIO_CONSEC_FRAMES:
        #sound.play()
#         print('DETECTED!!')
        draw_label(frame, "Drowsiness detected", (500,30), (0, 0, 255))
        pygame.mixer.music.load("drowsiness_alert.mp3")
        pygame.mixer.music.play()
        # cv2.putText(frame, "You are Drowsy", (150,200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 2)
#     else:
#         pygame.mixer.music.stop()
#         f_cnt = 0
    prev_yawn_status=yawn_status

    cv2.imshow('Live Landmarks', image_landmarks )
    cv2.imshow('Yawn Detection', frame )
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

C:\Users\shuva\anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


0
1
2
3
4
0
1
2
3
4
5
6
7
8
9
10
11
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
1
2
0
1
2
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
3
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
2
3
0
0
0
0
1
2
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
2
3
0
0
0
0
1
2
0
0
0
1
2
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
3
0
0
0
1
2
3
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 